# Majorana Operator 

In [1]:
from symred.symplectic_form import MajoranaOp
import numpy as np

# notes:

https://arxiv.org/pdf/2101.09349.pdf (pg11)

https://arxiv.org/abs/2110.10280

The Majorana operators {γ0, γ1, . . . , γm−1}, for m even, are linear Hermitian operators acting on the
fermionic Fock space 

$$H_{m/2} = \{ |b\rangle : b ∈ F_{2}^{m/2} \}$$

or equivalently the $m/2$-qubit complex Hilbert space satisfying $\forall 0 \leq i < j \leq m-1$:

1. $\gamma_{i}^{2} = \mathcal{I}$ - self inverse!
2. $\gamma_{i}\gamma_{j} = -\gamma_{j}\gamma_{i}$ - anti-commute!


These M single-mode operators generate a basis (up to phase factors) for the full algebra of Majorana operators via arbitrary products (https://arxiv.org/pdf/1908.08067.pdf pg9):

$$\gamma_{A} = \prod_{k \in A}^{M-1} \gamma_{k}$$

where $A \subseteq \{0,1,...,M-1 \} $ and represents the "support" of $\gamma_{A}$. We write this as $|A|$ where is the hamming weight of $\gamma_{A}$.

The anticommutator between two arbitrary Majorana operators $\gamma_{A}$ and $\gamma_{B}$ is determined by their individual supports and their overlap:



$$\{ \gamma_{A}, \gamma_{B} \} = \Big(1 + (-1)^{|A|\dot|B|- |A\cap B|} \Big) \gamma_{A}\gamma_{B}$$

Therefore:
- if $|A|\dot|B|- |A\cap B| = 0$
    - then terms anticommute
    
-else $|A|\dot|B|- |A\cap B| = 1$
    - and terms commute


# sympletic form

Class stores Majorana operator as a symplectic array and vector of coefficients.

rows of symplectic array give an individual operator and associated coefficient in coeff vec gives coefficient

therefore symplectic matrix is size $N \times M$ for $N$ terms and $M$ fermionic sites (note $M$ always even!)

In [2]:
operators = [
            [1,2,3,4], # op1
            [5] # op2
           ]
coeffs = [1,
         2]
Maj = MajoranaOp(operators, coeffs)
print(Maj)

(1+0j) γ1 γ2 γ3 γ4 +
(2+0j) γ5


In [3]:
Maj.symp_matrix

array([[0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1]])

In [4]:
Maj.coeff_vec

array([1.+0.j, 2.+0.j])

# NOTE order matters!

code uses bubble sort to fix operator to normal form

In [29]:
ops = [
            [4,3]# op1
           ]
amps = [1]

Maj = MajoranaOp(ops, amps)
print(Maj)

# order flipped! generating a sign!

(-1+0j) γ3 γ4


# check commutation relations

uses above definition!

### 1. termwise commutation!

In [5]:
operators = [
            [1,2,3,4], # op1
            [5,6,10] # op2
           ]
coeffs = [1,
         2]
Maj1 = MajoranaOp(operators, coeffs)


operators2 = [
            [0], # op1
            [2], # op2
            [3], # op3
           ]
coeffs2 = [1,
           2,
           3+1j]
Maj2 = MajoranaOp(operators2, coeffs2)

In [6]:
Maj1.commutes_termwise(Maj2)

array([[1, 0, 0],
       [0, 0, 0]])

In [7]:
print(Maj1)
# can see no overlap of terms therefore do (4*3)%2 = 0... therefore terms anticommute!
Maj1.adjacency_matrix()

(1+0j) γ1 γ2 γ3 γ4 +
(2+0j) γ5 γ6 γ10


array([[1, 1],
       [1, 1]])

In [8]:
print(Maj2)
# can see no overlap of terms therefore do (1*1)%2 = 1... therefore terms commute!
Maj2.commutes_termwise(Maj2)

(1+0j) γ0 +
(2+0j) γ2 +
(3+1j) γ3


array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [9]:
operator = [
    [0,1,2,3],
    [5,6],
    [3,5],
    [0,1,2,3,4,5,6,7],
    [2,3],
    [4]
]

coeffs = np.arange(2,len(operator)+2)

###
M = MajoranaOp(operator, coeffs)
print(M)

M.adjacency_matrix()

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4


array([[1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 1, 1]])

In [10]:
print(M*M)
print()

C = M.to_OF_op()
out = C*C
print(out)

(17+0j) I +
(42+0j) γ4 γ5 γ6 +
(20+0j) γ4 γ5 γ6 γ7 +
(-56+0j) γ3 γ4 γ5 +
(36+0j) γ2 γ3 γ5 γ6 +
(84+0j) γ2 γ3 γ4 +
(-24+0j) γ0 γ1 +
(-60+0j) γ0 γ1 γ4 γ5 γ6 γ7 +
(40+0j) γ0 γ1 γ2 γ4 γ6 γ7 +
(12+0j) γ0 γ1 γ2 γ3 γ5 γ6 +
(28+0j) γ0 γ1 γ2 γ3 γ4 +
(-30+0j) γ0 γ1 γ2 γ3 γ4 γ7

(17+0j) () +
(-24+0j) (0, 1) +
(28+0j) (0, 1, 2, 3, 4) +
(-30+0j) (0, 1, 2, 3, 4, 7) +
(12+0j) (0, 1, 2, 3, 5, 6) +
(40+0j) (0, 1, 2, 4, 6, 7) +
(-60+0j) (0, 1, 4, 5, 6, 7) +
(84+0j) (2, 3, 4) +
(36+0j) (2, 3, 5, 6) +
(-56+0j) (3, 4, 5) +
(42+0j) (4, 5, 6) +
(20+0j) (4, 5, 6, 7)


In [11]:
M.adjacency_matrix()

array([[1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 1, 1]])

# get basis for operator

In [12]:
from symred.utils import gf2_basis_for_gf2_rref, gf2_gaus_elim

In [13]:
ZX_symp = M.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0]])

In [14]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6


In [16]:
print(M)
print()
print(basis_op)

openF_M_op = M.to_OF_op()
basis_op_openF =  basis_op.to_OF_op()
print('commmutes: ', openF_M_op*basis_op_openF == basis_op_openF*openF_M_op)

print(M.commutes_termwise(basis_op))
M.commutes(basis_op)

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6
commmutes:  True
[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


True

In [17]:
print(M)

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4


# convert from Fermions to Majoranas

In [25]:
from openfermion import FermionOperator, get_majorana_operator
from symred.symplectic_form import convert_openF_fermionic_op_to_maj_op

ham = (FermionOperator('0^ 3', .5) +
       FermionOperator('3^ 0', 0.5) +
      FermionOperator('3^ 2^ 0 1', 0.5))

M_out = convert_openF_fermionic_op_to_maj_op(ham)

print(M_out.to_OF_op() == get_majorana_operator(ham))

True


In [26]:
print(M_out)

-0.25j γ1 γ6 +
(-0.03125+0j) γ1 γ3 γ5 γ7 +
-0.03125j γ1 γ3 γ5 γ6 +
-0.03125j γ1 γ3 γ4 γ7 +
(0.03125+0j) γ1 γ3 γ4 γ6 +
0.03125j γ1 γ2 γ5 γ7 +
(-0.03125+0j) γ1 γ2 γ5 γ6 +
(-0.03125+0j) γ1 γ2 γ4 γ7 +
-0.03125j γ1 γ2 γ4 γ6 +
0.25j γ0 γ7 +
0.03125j γ0 γ3 γ5 γ7 +
(-0.03125+0j) γ0 γ3 γ5 γ6 +
(-0.03125+0j) γ0 γ3 γ4 γ7 +
-0.03125j γ0 γ3 γ4 γ6 +
(0.03125+0j) γ0 γ2 γ5 γ7 +
0.03125j γ0 γ2 γ5 γ6 +
0.03125j γ0 γ2 γ4 γ7 +
(-0.03125+0j) γ0 γ2 γ4 γ6


In [27]:
print(basis_op, '\n')
op1 = MajoranaOp([[],[0]],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op1_dag = MajoranaOp([[],[0]],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot1 = op1*basis_op*op1_dag
print(rot1)

op1.commutes_termwise(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6 

(1+0j) γ2 γ3 γ5 γ6 +
1j γ1


array([[1, 1],
       [0, 1]])

In [ ]:
print(op1*op1_dag)

In [ ]:
print(op1.commutes(basis_op))

op1.commutes_termwise(basis_op)

In [ ]:
print(rot1)
print(op2)

In [ ]:
op2 = MajoranaOp([[], [1, 6]],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op2_dag = MajoranaOp([[], [1, 6]],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot2 = op2*rot1*op2_dag
print(rot2)

rot1.commutes_termwise(pp)

In [ ]:
print(op1*basis_op*op1_dag)

In [ ]:
# bug fixed
op2 = MajoranaOp([[12, 10]], [1])
print(op2)
from openfermion import MajoranaOperator
test = MajoranaOperator(term=(12,10))
print(test)

In [ ]:
op1 = MajoranaOp([[1,2,3,4,5]],[1])
op2 = MajoranaOp([[6]],[1])

print(op1.commutes(op2))

print(op1*op2*op1)